In [2]:
#Initialize dependencies
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorboard as tb
BASE_FOLDER = '/Users/oblivisheee/Documents/GitHub/novelsdreamer-ru-t4m/'
os.chdir(BASE_FOLDER)
from modules.transformer_custom import Transformer
from modules.regularization import RegularizedDenseLayer
from modules.create_config import transformer_config, metrics_config
from modules.tf_text_postprocess import TextPostprocessor
from modules.datagen import DataGenerator

# Config training

In [3]:
# Configure the Transformer layer
config = transformer_config()
config_metrics = metrics_config()

embedding_layer = tf.keras.layers.Embedding(config["input_vocab_size"], config["d_model"])
regularized_layer = RegularizedDenseLayer(config["d_model"])

model = Transformer(config["num_layers"], config["d_model"], config["num_heads"], config["dff"],
                          config["input_vocab_size"], config["input_vocab_size"], pe_input=config["maximum_position_encoding"],
                          pe_target=config["maximum_position_encoding"], rate=config["dropout_rate"], embedding=embedding_layer, regularized_layer=regularized_layer)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='auto')
optimizer = tf.keras.optimizers.legacy.Adam(name='Adam',learning_rate=0.001,
                                     epsilon=1e-8, amsgrad=True,
                                     beta_1=0.9, beta_2=0.999)

metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name=config_metrics['accuracy_set']), 
           tf.keras.metrics.MeanSquaredError(name=config_metrics['mean_sq_error']), 
           tf.keras.metrics.Precision(thresholds=config_metrics['thresholds'], name=config_metrics['precision']),]

model.compile(optimizer=optimizer, loss=loss_object, metrics=metrics)



In [4]:
MAIN_DATASET_DIR = 'dataset'
TRAIN_DATASET_DIR = os.path.join(MAIN_DATASET_DIR, 'train')
VALID_DATASET_DIR = os.path.join(MAIN_DATASET_DIR, 'valid')

datagen = DataGenerator(TRAIN_DATASET_DIR, VALID_DATASET_DIR)
(train_english, train_russian), (valid_english, valid_russian) = datagen.generate()

Train data info: 2 classes, 2 samples
Valid data info: 2 classes, 2 samples


# Start training

In [5]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
session_name='novelsdreamer_test_1'
epochs = 50
model.fit_model(train_english=train_english, 
                         train_russian=train_russian,
                         valid_english=valid_english,
                         valid_russian=valid_russian,
                         epochs=epochs,
                         session_name=session_name,
                         shuffle=True,
                         save_model_each_epoch=True,
                         model_name='novelsdreamer-ru-t4m')

  0%|          | 0/50 [00:00<?, ?it/s]

INFO:tensorflow:Epoch 1 Loss 4.8430 Validation Loss 3.5827
INFO:tensorflow:Beam search result: [[[2], -1.3473692623799352], [[1], -0.3022289574713113]]
INFO:tensorflow:Epoch 1 finished.
INFO:tensorflow:Model of 1 epoch saved in novelsdreamer_test_1/results/trained_models/novelsdreamer-ru-t4m_epoch_1


  2%|▏         | 1/50 [00:23<19:31, 23.91s/it]

INFO:tensorflow:Epoch 2 Loss 1.9210 Validation Loss 0.0000
INFO:tensorflow:Beam search result: [[[1], -3.2977024190890813]]
INFO:tensorflow:Epoch 2 finished.
INFO:tensorflow:Model of 2 epoch saved in novelsdreamer_test_1/results/trained_models/novelsdreamer-ru-t4m_epoch_2


  4%|▍         | 2/50 [00:34<13:05, 16.37s/it]

...

INFO:tensorflow:Epoch 4 Loss 2.3109 Validation Loss 33.4782
INFO:tensorflow:Beam search result: [[[2], -3.7541423860737972], [[1], -2.221307107170497]]
INFO:tensorflow:Epoch 4 finished.
INFO:tensorflow:Model of 4 epoch saved in novelsdreamer_test_1/results/trained_models/novelsdreamer-ru-t4m_epoch_4


  8%|▊         | 4/50 [00:57<09:51, 12.87s/it]

INFO:tensorflow:Epoch 5 Loss 13.3467 Validation Loss 23.4362
INFO:tensorflow:Beam search result: [[[2], -3.511462423017941], [[1], -2.3086663608401246]]
INFO:tensorflow:Epoch 5 finished.
INFO:tensorflow:Model of 5 epoch saved in novelsdreamer_test_1/results/trained_models/novelsdreamer-ru-t4m_epoch_5


 10%|█         | 5/50 [01:08<10:19, 13.76s/it]


KeyboardInterrupt: 

In [ ]:
epochs = 50
train_dataset = tf.data.Dataset.from_tensor_slices((train_english.tolist(), train_russian.tolist()))
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_english.tolist(), valid_russian.tolist()))

model.fit(train_dataset, 
          validation_data=valid_dataset,
          epochs=epochs,
          verbose='auto',
          shuffle=True, workers=4, validation_freq=10)
